### Import Modules

In [196]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

matplotlib.rcParams['figure.figsize'] = [10, 10]

In [198]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Specify fit parameters

In [182]:
p_dict = {
    'order' : {
        'fit' : 'nnnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 8 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'ma-old', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    
    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
    
    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_prior' : False,
    'save_results' : False,
}

### Do fit

In [183]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print bootstrapper
    
if p_dict['save_prior']:
    print 'Saving prior...'
    data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Using default prior.
Making fits...
100% complete Time (s):  16.745000124
Compiling results...
Done.

Total time (s):  19.4849998951 



### Or make all fits

In [199]:
p_dict = {
    'order' : {
        'vol' : 8 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'

    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],

    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,

    'save_prior' : False,
    'save_results' : True,
}

In [200]:
for fit_type in ['xpt', 'xpt-taylor', 'ma', 'ma-taylor', 'ma-old']:
    for order_fit in ['nlo', 'nnlo', 'nnnlo']:
        
        print fit_type, order_fit
        
        # Specify fit parameters
        p_dict['order']['fit'] = order_fit
        p_dict['fit_type'] = fit_type
        
        # Do fit
        t0 = time.time()

        # Load data
        data_loader = dl.data_loader()
        fit_data = data_loader.get_fit_data()

        # Get prior
        prior = None
        if p_dict['use_prior']:
            prior = data_loader.get_prior(p_dict['fit_type'])

        # Make bootstrapper
        bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                                       fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

        if p_dict['make_plots']:
            data_loader.save_plots(
                bootstrapper.make_plots(
                    show_error_ellipses=p_dict['show_error_ellipses'], 
                    show_bootstrap_histograms=p_dict['show_bs_histograms']),
                output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
            )
        else:
            print bootstrapper

        if p_dict['save_prior']:
            print 'Saving prior...'
            data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])

        if p_dict['save_results']:
            data_loader.save_fit_info(bootstrapper.get_fit_info())


        t1 = time.time()

        print "\nTotal time (s): ", t1 - t0, "\n"

xpt nlo
Using default prior.
Making fits...
100% complete Time (s):  5.64100003242
Compiling results...
Done.
Saving...
Done.

Total time (s):  8.38900017738 

xpt nnlo
Using default prior.
Making fits...
100% complete Time (s):  11.7650001049
Compiling results...
Done.
Saving...
Done.

Total time (s):  14.7669999599 

xpt nnnlo
Using default prior.
Making fits...
100% complete Time (s):  19.2749998569
Compiling results...
Done.
Saving...
Done.

Total time (s):  22.2400000095 

xpt-taylor nlo
Using default prior.
Making fits...
100% complete Time (s):  5.45999979973
Compiling results...
Done.
Saving...
Done.

Total time (s):  8.4319999218 

xpt-taylor nnlo
Using default prior.
Making fits...
100% complete Time (s):  11.8489999771
Compiling results...
Done.
Saving...
Done.

Total time (s):  14.6180000305 

xpt-taylor nnnlo
Using default prior.
Making fits...
100% complete Time (s):  19.5249998569
Compiling results...
Done.
Saving...
Done.

Total time (s):  22.3990001678 

ma nlo
Using d

In [ ]:
#bootstrapper.fits[0].show_plots(view='std')

In [193]:
print bootstrapper._make_fit_data(0)

{'lam2_chi': array([0.328(40), 0.328(40), 0.328(40), 0.328(40), 0.624(75), 0.624(75),
       0.624(75), 0.624(75), 0.624(75), 0.624(75), 0.98(12), 0.98(12),
       0.98(12), 0.98(12)], dtype=object), 'mss': array([0.309362(74), 0.31169(11), 0.31244(11), 0.31327(11), 0.41421(11),
       0.41539(12), 0.41576(14), 0.41802(16), 0.41970(13), 0.42073(13),
       0.51496(13), 0.51876(16), 0.52008(22), 0.52193(22)], dtype=object), 'mrs': array([0.31126(21), 0.31500(29), 0.31598(53), 0.31589(61), 0.42721(34),
       0.42800(34), 0.42800(34), 0.43046(43), 0.4320(11), 0.43203(70),
       0.54166(54), 0.54898(74), 0.5528(17), 0.5538(17)], dtype=object), 'a2DI': array([0.00515(42), 0.00515(42), 0.00515(42), 0.00515(42), 0.0312(26),
       0.0312(26), 0.0312(26), 0.0312(26), 0.0312(26), 0.0312(26),
       0.087(11), 0.087(11), 0.087(11), 0.087(11)], dtype=object), 'mru': array([0.23284(39), 0.24688(51), 0.25278(61), 0.26040(81), 0.3261(14),
       0.33158(91), 0.33158(91), 0.34390(98), 0.3519(28), 0